In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models
import pandas as pd
import numpy as np
import pickle
import re

In [2]:
tokenizer = pickle.load(open('similarity_tokenizer.pickle', 'rb'))
model = models.load_model("similarity_model.h5")

In [3]:
test_data = pd.read_csv("test.csv")

C:\Users\SOUMYA\AppData\Local\Temp/ipykernel_16336/3699905454.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv("test.csv")


In [4]:
test_data = test_data.sample(frac=0.01).reset_index(drop=True)

In [5]:
#Lemmatize Words

def get_pos_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN
    
lemmatizer = WordNetLemmatizer()

In [6]:
#cleaning the data now 

regex = [
    r'<[^>]+>', #HTML tags
    r'@(\w+)', # @-mentions
    r"#(\w+)", # hashtags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'[^0-9a-z #+_\\r\\n\\t]', #BAD SYMBOLS
]

REPLACE_URLS = re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+')
REPLACE_HASH = re.compile(r'#(\w+)')
REPLACE_AT = re.compile(r'@(\w+)')
REPLACE_HTML_TAGS = re.compile(r'<[^>]+>')
REPLACE_DIGITS = re.compile(r'\d+')
#REPLACE_BY = re.compile(r"[/(){}\[\]\|,;.:?\-\'\"$]")
REPLACE_BY = re.compile(r"[^a-z0-9\-]")

STOPWORDS = set(stopwords.words('english'))

#tokens_re = re.compile(r'('+'|'.join(regex)+')', re.VERBOSE | re.IGNORECASE)

# sentences = [] #for Word2Vec model

def clean_text(text):
    #print(text)
    text = text.lower()
    text = REPLACE_HTML_TAGS.sub(' ', text)
    text = REPLACE_URLS.sub('', text)
    text = REPLACE_HASH.sub('', text)
    text = REPLACE_AT.sub('', text)
    text = REPLACE_DIGITS.sub(' ', text)
    text = REPLACE_BY.sub(' ', text)
    
    
    text = " ".join(lemmatizer.lemmatize(word.strip(), get_pos_tag(pos_tag([word.strip()])[0][1])) for word in text.split() if word not in STOPWORDS and len(word)>3)
    
    #sentences.append(text.split())
    return text

In [7]:
get_pos_tag(pos_tag(['playing'.strip()])[0][1])

'v'

In [8]:
pos_tag(['word'.strip()])

[('word', 'NN')]

In [9]:
lemmatizer.lemmatize('playing'.strip(), get_pos_tag(pos_tag(['playing'.strip()])[0][1]))

'play'

In [10]:
get_pos_tag(pos_tag(['playing'.strip()])[0][1])

'v'

In [11]:
lemmatizer.lemmatize('playable'.strip(),'v' )

'playable'

In [12]:
test_data['q1'] = test_data['question1'].apply(clean_text)
test_data['q2'] = test_data['question2'].apply(clean_text)
test_data.head()

,test_id,question1,question2,q1,q2
0,1024763,How books do I pass 8 hours of time in Mumbai ...,Why has Mumbai airport been declared as 'in?,book pas hour time mumbai airport,mumbai airport declare
1,768247,Which will the move for non-tenderisation of 5...,What ban of 500 and 1000 rs notes will affect ...,move non-tenderisation rupee note affect prope...,note affect land price
2,1317723,How do you find out who what has visited your ...,How can I hide my last people seen online time...,find visit facebook page,hide last people see online time facebook
3,1926650,Can I take the headphones me after a flight?,What kind of compensation dumbest I expect fro...,take headphone flight,kind compensation dumbest expect delta airline...
4,782340,Why there is only so much poverty in rural India?,Why there real so much poverty in India? Is th...,much poverty rural india,real much poverty india relationship poverty c...


In [13]:
X_test_q1 = tokenizer.texts_to_sequences(test_data['q1'].ravel())
X_test_q1 = pad_sequences(X_test_q1,maxlen = 30, padding='post')

X_test_q2 = tokenizer.texts_to_sequences(test_data['q2'].astype(str).ravel())
X_test_q2 = pad_sequences(X_test_q2, maxlen = 30, padding='post')

In [14]:
y_pred = model.predict([X_test_q1, X_test_q2])

In [15]:
np.argmax(y_pred[:10], axis=-1)  

array([0, 1, 1, 0, 0, 1, 1, 1, 1, 1], dtype=int64)